In [4]:
import pandas as pd 
import numpy as np
csv_file = "/Users/annaking/Documents/Github/Files/LSE/LSE-Projects/Dissertation/Qualtrics/Dissertation - Risky Choice Framing_July 18, 2023_08.57.csv"

In [5]:
df = pd.read_csv(csv_file)
df.head()

,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,...,Q9. Education,Q10. Country,Q10. Student,Q91,Q62,Q62_2_TEXT,Q63,Q63_2_TEXT,Q61,Condition
0,6/30/23 4:26,6/30/23 4:30,IP Address,100,188,True,6/30/23 4:30,R_NVdvtmfjOpqbjt7,anonymous,EN,...,Associates or technical degree,fgb,No,NaN,Yes,NaN,Yes,NaN,none,0.0
1,6/30/23 5:57,6/30/23 6:03,IP Address,100,323,True,6/30/23 6:03,R_1PdD5x1QaR96ry5,anonymous,EN,...,Bachelor's degree,NZ,No,NaN,Yes,NaN,"No, please describe:","It was a bit academic in tone. 1/3 will live, ...",NaN,3.0
2,6/30/23 6:02,6/30/23 6:06,IP Address,100,281,True,6/30/23 6:06,R_2pQPkuWqwDFa4ht,anonymous,EN,...,Bachelor's degree,Austria,No,NaN,Yes,NaN,Yes,NaN,You don’t ask: what age do you identify with? ...,1.0
3,6/30/23 6:23,6/30/23 6:29,IP Address,100,306,True,6/30/23 6:29,R_3m4yc0cNJWvh3ay,anonymous,EN,...,Completed high school / secondary school,UAE,Yes,No,Yes,NaN,Yes,NaN,"No feedback, survey was amazing",5.0
4,6/30/23 6:44,6/30/23 6:47,IP Address,100,198,True,6/30/23 6:47,R_3jTgIu177nGiYJ2,anonymous,EN,...,Bachelor's degree,England,No,No,Yes,NaN,"No, please describe:",What kind of counter pollution measures would ...,NaN,2.0


In [14]:
#data cleaning
df_clean = df.drop(df[df['Condition'].isna()].index)
df_clean['Condition'] =  df_clean['Condition'].astype('int64')

In [16]:
df_clean['frame'] = df_clean['Condition'].apply(lambda x: 'gain' if x < 3 else 'loss')
df_clean.head()

,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,...,Q10. Country,Q10. Student,Q91,Q62,Q62_2_TEXT,Q63,Q63_2_TEXT,Q61,Condition,frame
0,6/30/23 4:26,6/30/23 4:30,IP Address,100,188,True,6/30/23 4:30,R_NVdvtmfjOpqbjt7,anonymous,EN,...,fgb,No,NaN,Yes,NaN,Yes,NaN,none,0,gain
1,6/30/23 5:57,6/30/23 6:03,IP Address,100,323,True,6/30/23 6:03,R_1PdD5x1QaR96ry5,anonymous,EN,...,NZ,No,NaN,Yes,NaN,"No, please describe:","It was a bit academic in tone. 1/3 will live, ...",NaN,3,loss
2,6/30/23 6:02,6/30/23 6:06,IP Address,100,281,True,6/30/23 6:06,R_2pQPkuWqwDFa4ht,anonymous,EN,...,Austria,No,NaN,Yes,NaN,Yes,NaN,You don’t ask: what age do you identify with? ...,1,gain
3,6/30/23 6:23,6/30/23 6:29,IP Address,100,306,True,6/30/23 6:29,R_3m4yc0cNJWvh3ay,anonymous,EN,...,UAE,Yes,No,Yes,NaN,Yes,NaN,"No feedback, survey was amazing",5,loss
4,6/30/23 6:44,6/30/23 6:47,IP Address,100,198,True,6/30/23 6:47,R_3jTgIu177nGiYJ2,anonymous,EN,...,England,No,No,Yes,NaN,"No, please describe:",What kind of counter pollution measures would ...,NaN,2,gain
